In [1]:
import pickle
import numpy as np
from sklearn.model_selection import train_test_split, GridSearchCV
from sklearn.linear_model import LogisticRegression
from sklearn.ensemble import RandomForestClassifier
from sklearn.neighbors import KNeighborsClassifier
from sklearn.preprocessing import StandardScaler
from sklearn.decomposition import PCA
from sklearn.pipeline import Pipeline
from sklearn.linear_model import LogisticRegressionCV
from sklearn.metrics import roc_auc_score


In [2]:
# Load data
X, y = pickle.load(open('listens.p', 'rb'))

## 4.1 Data Exploration

### (a) How many users and songs are there in the data?

In [3]:
no_users = X.shape[0]
no_songs = X.shape[1]
print("Number of users:", no_users)
print("Number of songs:", no_songs)

Number of users: 1960
Number of songs: 990


### (b) What is the maximum number of times that a user has listened to one song?

In [4]:
max_listens = np.max(X)
print("Maximum number of times that a user has listened to one song:", max_listens)

Maximum number of times that a user has listened to one song: 214.0


### (c) What percentage of users have listened to the special song?


In [5]:
special_song_listen = np.sum(y)
special_song_listen_percent = (special_song_listen / no_users) * 100
print("Percentage of users that have listened to the special song:", special_song_listen_percent)

Percentage of users that have listened to the special song: 16.3265306122449


### (d) Count the number of times each song has been listened to. What’s the smallest number? The largest number? The average?

In [6]:
song_listens = np.sum(X, axis=0)
smallest_songs = np.min(song_listens)
largest_songs = np.max(song_listens)
avg_songs = np.mean(song_listens)
print("Total number of times each song has been listened to:", sum(song_listens))
print("Smallest number of times each song has been listened to.:", smallest_songs)
print("Highest number of times each song has been listened to:", largest_songs)
print("Average number of times each song has been listened to:", avg_songs)

Total number of times each song has been listened to: 60991.0
Smallest number of times each song has been listened to.: 6.0
Highest number of times each song has been listened to: 688.0
Average number of times each song has been listened to: 61.607070707070704


### (e) Count the number of users that have listened to each song. What’s the smallest number? The largest number? The average?

In [7]:
users_per_song = np.sum(X > 0, axis=0)
min_users_per_song = np.min(users_per_song)
max_users_per_song = np.max(users_per_song)
avg_users_per_song = np.mean(users_per_song)
print("Smallest number of users that have listened to each song:", min_users_per_song)
print("Larget number of users that have listened to each song:", max_users_per_song)
print("Average number of users that have listened to each song:", avg_users_per_song)

Smallest number of users that have listened to each song: 6
Larget number of users that have listened to each song: 340
Average number of users that have listened to each song: 32.07878787878788


### (f) Count the total number of listens by each user. What’s the smallest number? The largest number? The average?

In [8]:
# (f) Count the total number of listens by each user.
listens_per_user = np.sum(X, axis=1)
smallest_listens_per_user = np.min(listens_per_user)
largest_listens_per_user = np.max(listens_per_user)
avg_listens_per_user = np.mean(listens_per_user)
print("Total number of listens by each user", song_listens)
print("Total number of listens by all users", sum(song_listens))
print("Smallest number of listens by each user:", smallest_listens_per_user)
print("Largest number of listens by each user:", largest_listens_per_user)
print("Average number of listens by each user:", avg_listens_per_user)

Total number of listens by each user [ 33.  56.  34.  11.  45.  40.  14. 131.  19.  12.   8.   9.  53.  12.
 409. 159.  94.  24.  14.  13.  10.  10.  50. 108. 116.  10. 103. 143.
  52.  96. 123. 377. 481.  13. 297. 376. 142. 150.  92.  43. 688.  81.
 220. 156.  33. 601. 480.  10.  22. 163.  52. 100. 584. 158. 103.  15.
 180. 282.  56.  86.  97.  44. 249. 307.  64. 209.  66. 353.  39. 419.
 419. 205. 157.  18. 335.  69. 289. 403. 269. 163. 151. 167. 246. 103.
 121. 356. 234. 335. 322. 111.  88.   9. 128. 131. 102.  17.  50. 132.
 106. 211. 155.   6.  38.  75. 334.  36.   7. 152.  95.  25.  13. 132.
  71.  44.  40. 158.  15.  98. 149. 115.  88.  87.  73. 238. 183. 478.
 242. 247. 413.  85.  11. 264. 235.  17.  18.  12.  40.  41.  17.  23.
  22. 141.  27.  40.  74.  71.  37. 131.  34.  10. 107.  79. 111.  16.
  62. 115.  61. 176. 186.  25.  70.  37. 515.  84.  36.  32. 152.  69.
  23.  16. 228. 186. 418.  59. 121.  26.  40.  16.  33. 125.  10.  17.
  16.  56. 320. 205. 147. 246.  73. 345.

### (g) Count the number of songs listened to by each user. What’s the smallest number? The largest number? The average?

In [9]:
songs_per_user = np.sum(X > 0, axis=1)
smallest_songs_per_user = np.min(songs_per_user)
largest_songs_per_user = np.max(songs_per_user)
avg_songs_per_user = np.mean(songs_per_user)
print("Smallest number of songs listened to by each user:", smallest_songs_per_user)
print("Largest number of songs listened to by each user:", largest_songs_per_user)
print("Average number of songs listened to by each user:", avg_songs_per_user)

Smallest number of songs listened to by each user: 2
Largest number of songs listened to by each user: 138
Average number of songs listened to by each user: 16.203061224489797


## 4.2 Baseline (No data pre-processing): We will treat the first 1400 rows as the training set, and the remaining rows as the test set. For each of the algorithms below, report the out-of-sample AUC.

In [10]:
# Splitting the data into train and test sets
X_train = X[:1400]
X_test = X[1400:]

y_train = y[:1400]
y_test = y[1400:]

# Verify the shapes of the train and test sets
print("Shape of X_train:", X_train.shape)
print("Shape of X_test:", X_test.shape)
print("Shape of y_train:", y_train.shape)
print("Shape of y_test:", y_test.shape)

Shape of X_train: (1400, 990)
Shape of X_test: (560, 990)
Shape of y_train: (1400,)
Shape of y_test: (560,)


### (a) Logistic Regression with L1 penalty selected via 5-fold CV

In [13]:
# Create a Logistic Regression model with L1 penalty and 5-fold cross-validation
log_reg = LogisticRegressionCV(cv=5, penalty='l1', solver='liblinear', scoring='roc_auc', random_state=42)

# Train the model on the training data
log_reg.fit(X_train, y_train)

# Predict probabilities for the test set
y_pred_proba = log_reg.predict_proba(X_test)[:, 1]

# Calculate the out-of-sample AUC
auc = roc_auc_score(y_test, y_pred_proba)
print("Out-of-sample AUC:", auc)


Out-of-sample AUC: 0.7775266638462738


### b) Random Forest

In [12]:

# Creating the model
model_rf = RandomForestClassifier()

# # Setting parameters for grid search
grid_rf = {'n_estimators': [100, 200, 300], 'max_depth': [None, 5, 10]}

# Creating GridSearchCV object with 5-fold cross-validation and ROC AUC scoring
cv_rf = GridSearchCV(model_rf, grid_rf, cv=5, scoring='roc_auc')

# Fitting the model on the training data
cv_rf.fit(X_train, y_train)

# Calculating out-of-sample AUC using the best estimator from grid search
auc_rf = roc_auc_score(y_test, cv_rf.predict_proba(X_test)[:, 1])
print("Out-of-sample AUC for Random Forest:", auc_rf)


Out-of-sample AUC for Random Forest: 0.8238516176437843


### (c) k-NN with both k and the distance function (try euclidean, manhattan, and hamming) selected via 5-fold CV

In [15]:
# Define the parameter grid
param_grid = {'n_neighbors': [3, 5, 7, 9], 'metric': ['euclidean', 'manhattan', 'hamming']}

# Create k-NN classifier
knn = KNeighborsClassifier()

# Grid search with 5-fold cross-validation
grid_search = GridSearchCV(knn, param_grid, cv=5, scoring='roc_auc')

# Fit the grid search to the data
grid_search.fit(X_train, y_train)

# Predict probabilities for the test set
y_pred_proba_knn = grid_search.predict_proba(X_test)[:, 1]

# Calculate the out-of-sample AUC
auc_knn = roc_auc_score(y_test, y_pred_proba_knn)
print("Out-of-sample AUC for k-NN:", auc_knn)

Out-of-sample AUC for k-NN: 0.7301997283516288


## 4.3: Column Normalization: Repeat part 4.2, this time pre-processing the data beforehand by normalizing the columns

In [15]:
scaler = StandardScaler()
X_train_normalized = scaler.fit_transform(X_train)
X_test_normalized = scaler.transform(X_test)

In [16]:
# Create a Logistic Regression model with L1 penalty and 5-fold cross-validation
log_reg_normalized = LogisticRegressionCV(cv=5, penalty='l1', solver='liblinear', scoring='roc_auc', random_state=42)

# Train the model on the normalized training data
log_reg_normalized.fit(X_train_normalized, y_train)

# Predict probabilities for the normalized test set
y_pred_proba_normalized = log_reg_normalized.predict_proba(X_test_normalized)[:, 1]

# Calculate the out-of-sample AUC for the normalized dataset
auc_normalized = roc_auc_score(y_test, y_pred_proba_normalized)
print("Out-of-sample AUC for Logistic Regression with normalized data:", auc_normalized)

/Users/panktiantani/anaconda3/lib/python3.11/site-packages/sklearn/svm/_base.py:1242: ConvergenceWarning: Liblinear failed to converge, increase the number of iterations.
  warnings.warn(
/Users/panktiantani/anaconda3/lib/python3.11/site-packages/sklearn/svm/_base.py:1242: ConvergenceWarning: Liblinear failed to converge, increase the number of iterations.
  warnings.warn(
/Users/panktiantani/anaconda3/lib/python3.11/site-packages/sklearn/svm/_base.py:1242: ConvergenceWarning: Liblinear failed to converge, increase the number of iterations.
  warnings.warn(
/Users/panktiantani/anaconda3/lib/python3.11/site-packages/sklearn/svm/_base.py:1242: ConvergenceWarning: Liblinear failed to converge, increase the number of iterations.
  warnings.warn(
/Users/panktiantani/anaconda3/lib/python3.11/site-packages/sklearn/svm/_base.py:1242: ConvergenceWarning: Liblinear failed to converge, increase the number of iterations.
  warnings.warn(


In [ ]:
# Creating the Random Forest Model
model_rf_normalized = RandomForestClassifier()

# Setting parameters for grid search
grid_rf = {'n_estimators': [100, 200, 300], 'max_depth': [None, 5, 10]}

# Creating GridSearchCV object with 5-fold cross-validation and ROC AUC scoring
cv_rf_normalized = GridSearchCV(model_rf_normalized, grid_rf, cv=5, scoring='roc_auc')

# Fitting the model on the normalized training data
cv_rf_normalized.fit(X_train_normalized, y_train)

# Calculating out-of-sample AUC using the best estimator from grid search
auc_rf_normalized = roc_auc_score(y_test, cv_rf_normalized.predict_proba(X_test_normalized)[:, 1])
print("Out-of-sample AUC for Random Forest with normalized data:", auc_rf_normalized)

Random Forest AUC (Column Normalization): 0.7905254816502402


In [ ]:
# Define the parameter grid
param_grid = {'n_neighbors': [3, 5, 7, 9], 'metric': ['euclidean', 'manhattan', 'hamming']}

# Create k-NN classifier
knn = KNeighborsClassifier()

# Grid search with 5-fold cross-validation
grid_search = GridSearchCV(knn, param_grid, cv=5, scoring='roc_auc')

# Fit the grid search to the normalized data
grid_search.fit(X_train_normalized, y_train)

# Predict probabilities for the normalized test set
y_pred_proba_knn_normalized = grid_search.predict_proba(X_test_normalized)[:, 1]

# Calculate the out-of-sample AUC for the normalized dataset
auc_knn_normalized = roc_auc_score(y_test, y_pred_proba_knn_normalized)
print("Out-of-sample AUC for k-NN with normalized data:", auc_knn_normalized)

k-NN AUC (Column Normalization): 0.670571046505869


## 5.4 PCA

In [ ]:
# Defining pipeline with PCA and k-NN
pipeline = Pipeline([('pca', PCA()), ('knn', KNeighborsClassifier())])

# Grid search with pipeline
param_grid = {
    'pca__n_components': [10, 20, 30, 40, 50],
    'knn__n_neighbors': [3, 5, 7, 9],
    'knn__metric': ['euclidean', 'manhattan', 'hamming']
}

# Using GridSearchCV to find best parameters within specified ranges
grid_search = GridSearchCV(pipeline, param_grid, cv=5, scoring='roc_auc').fit(X_train, y_train)

# Evaluating the best model found by GridSearchCV on the test set
y_pred_proba_test = grid_search.predict_proba(X_test)[:, 1]
auc_score_test = roc_auc_score(y_test, y_pred_proba_test)

# Printing best parameters and out-of-sample AUC score
print("Best parameters:", grid_search.best_params_)
print(f"Out-of-sample AUC Score with PCA and k-NN: {auc_score_test:.3f}")

PCA + k-NN AUC: 0.6062602149472537


## 4.5 PCA and Column Normalization:

In [ ]:
# Defining pipeline with PCA, normalization, and logistic regression
pipeline_log_reg = Pipeline([
    ('pca', PCA(random_state=1)),
    ('std', StandardScaler()),
    ('logreg', LogisticRegression(penalty='l1', solver="saga", random_state=123, max_iter=100000, n_jobs=-1))
])

# Performing grid search
param_grid_log_reg = {
    'pca__n_components': [i*10 for i in range(1, 20)],
    'logreg__C': np.logspace(-3, 1, 12)
}

grid_search_log_reg = GridSearchCV(pipeline_log_reg, param_grid=param_grid_log_reg, cv=5, scoring='roc_auc', verbose=1, n_jobs=-1)
grid_search_log_reg.fit(X_train, y_train)

# Evaluating model
y_pred_proba_log_reg_test = grid_search_log_reg.predict_proba(X_test)[:, 1]
auc_score_log_reg_test = roc_auc_score(y_test, y_pred_proba_log_reg_test)

# Printing best parameters and AUC score
print("Best parameters for Logistic Regression:", grid_search_log_reg.best_params_)
print("ROC AUC score for Logistic Regression with L1 penalty selected via 5-fold CV (PCA, Standardized):", auc_score_log_reg_test)


Logistic Regression AUC (PCA + Column Normalization): 0.72344113714031


In [ ]:
# Defining pipeline with PCA, normalization, and Random Forest
pipeline_rf = Pipeline([
    ('pca', PCA()),
    ('scaler', StandardScaler()),
    ('rf', RandomForestClassifier())
])

# Setting up parameter grid for PCA components
param_grid_rf = {'pca__n_components': [10, 20, 30, 40, 50]}

# Performing grid search
grid_search_rf = GridSearchCV(pipeline_rf, param_grid_rf, cv=5, scoring='roc_auc').fit(X_train, y_train)

# Evaluating model
y_pred_proba_rf_test = grid_search_rf.predict_proba(X_test)[:, 1]
auc_score_rf_test = roc_auc_score(y_test, y_pred_proba_rf_test)

# Printing best parameters and out-of-sample AUC score
print("Best parameters for Random Forest:", grid_search_rf.best_params_)
print(f"Out-of-sample AUC Score with PCA, Normalization, and Random Forest: {auc_score_rf_test:.3f}")

Random Forest AUC (PCA + Column Normalization): 0.734485661928582


In [ ]:
# Defining pipeline with PCA, normalization, and k-NN
pipeline_knn = Pipeline([
    ('pca', PCA()),
    ('scaler', StandardScaler()),
    ('knn', KNeighborsClassifier())
])

# Setting up parameter grid for PCA components and k-NN parameters
param_grid_knn = {
    'pca__n_components': [10, 20, 30, 40, 50],
    'knn__n_neighbors': [3, 5, 7, 9, 47],
    'knn__metric': ['euclidean', 'manhattan', 'hamming']
}

# Performing grid search
grid_search_knn = GridSearchCV(pipeline_knn, param_grid_knn, cv=5, scoring='roc_auc').fit(X_train, y_train)

# Evaluating model
y_pred_proba_knn_test = grid_search_knn.predict_proba(X_test)[:, 1]
auc_score_knn_test = roc_auc_score(y_test, y_pred_proba_knn_test)

# Printing best parameters and out-of-sample AUC score
print("Best parameters for k-NN:", grid_search_knn.best_params_)
print(f"Out-of-sample AUC Score with PCA, Normalization, and k-NN: {auc_score_knn_test:.3f}")

k-NN AUC (PCA + Column Normalization): 0.7038532019216484
